### Doctor's duty roster###
This purpose of this app is to automatically assign OFM shifts to doctors presented at hospital during that month.
(OFM stands for occupational-family-medicine). The OFM shifts are between 8:30-16:30, every weekday except for Tuesday.

[ลิงค์วันหยุด](https://hilight.kapook.com/view/188249)

In [1]:
# change var
YEAR = 2020
MONTH = 12
HOLIDAYS = [10,11,31] #ไม่ต้องใส่เสาร์อาทิตย์
SAVE = "December" #this month
PREVIOUS_SAVE = "November"
# remove resigned and not present or คนที่ออก OPD อื่นทั้งเดือน
RESIGNED = ['ศุภาพิชญ์','พุฒิพงศ์'] 
NOT_PRESENT = ["ภาณุวัฒน์", "ปรียาภรณ์"]
OTHER_OPD = [] 

# คนที่ไม่ว่างในแต่ละวัน
# สามารถเขียนได้ 2 วิธี
# ชื่อภาค ("ENT","Eye" ลองดูใน excel หรือดูตารางข้างล่างว่าพิมพ์ว่าอะไร)  ตามด้วย list วันของสัปดาห์ ["monday","tuesday"..] แล้วคร่อมทั่งหมดด้วยวงเล็บ (กลายเป็น tuple)
# เช่น ('ENT',['thursday', 'friday'])
# ลิสต์ของคน ['ปรียาภรณ์,'นภดล'] ตามด้วยลิสต์ของวันในสัปดาห์ ["monday","tuesday"..] หรือวันที่ [11,12,13,14] ซึ่งวันที่นี้สามารถเขียนอีกแบบได้คือ [*np.arange(11,14+1)] = วันที่ 11-14 แล้วคร่อมทั่งหมดด้วยวงเล็บ (กลายเป็น tuple)
# เช่น (['เอนกโชค'], [*np.arange(9,13+1), *np.arange(25,31+1)]) -- ลา 9-13 กับ 25-31
# หรือ (['ปรียาภรณ์',"นภดล"], ['wednesday',"friday"])

# ในทุกเดือน จะมี med กับ ENT ที่มีการฟิกวันไม่ว่าง คือต้องออก OPD ของตัวเอง: ENT ไม่ว่าง พฤ, เกลือไม่ว่างพุธ, Ckits ไม่ว่าง จันทร์พฤ, 'ปรียาภรณ์' ไม่ว่างพุธศุกร์, นภดล ไม่ว่างอังคารพฤ
# อย่าลืม ',' ระหว่างบรรทัด

UNAVAILABLE = [
   ('ENT',['thursday']) ,
   (['ภาณุวัฒน์'],['wednesday']),
   (['ศิรวิชญ์'], ['monday',"thursday"] ),
   (['ปรียาภรณ์'], ['wednesday',"friday"] ),
   (['นภดล'], ['tuesday',"thursday"]) ,
]

UNAVAILABLE

[('ENT', ['thursday']),
 (['ภาณุวัฒน์'], ['wednesday']),
 (['ศิรวิชญ์'], ['monday', 'thursday']),
 (['ปรียาภรณ์'], ['wednesday', 'friday']),
 (['นภดล'], ['tuesday', 'thursday'])]

In [1]:
#1. Get the dates of every weekdays of that month accordingly - assign it to a dictionary *weekday_date*
#2. Create a list with all the dates of that month - *all_dates*

In [18]:
from datetime import date, timedelta
import pandas as pd
import numpy as np

weekday_dict = {0:'monday', 1:'tuesday', 2:'wednesday', 3:'thursday', 4:'friday', 5:'saturday', 6:'sunday'}
weekday_date = {weekday:[] for weekday in weekday_dict.values()}
Thai_dict = {'january':"มกราคม",'february':"กุมภาพันธ์",'march':"มีนาคม",'may':"พฤษภาคม",'june':"มิถุนายน",\
             'july':"กรกฎาคม",'august':"สิงหาคม",'september':"กันยายน",'october':"ตุลาคม",'november':"พฤศจิกายน",\
             'december':"ธันวาคม"}

def get_dates (year, month):
    d = date(year, month, 1) #1 = first day of the month, do not change
    while d.month == month:
        day_of_week = weekday_dict[d.weekday()]
        weekday_date[day_of_week].append(d)
        d += timedelta(days = 1)
    return weekday_date

"""Change every month"""
weekday_date = get_dates(YEAR,MONTH)


all_dates = []
for week_dates in weekday_date.values():
    all_dates = all_dates + [i for i in week_dates]
all_dates.sort()

In [19]:
# weekday_date

In [20]:
# all_dates

### create date object for the holidays we wish to exclude

In [5]:
holidays = []
def create_holidays(year, month, day):
    d = date(year, month, day)
    holidays.append(d)
    return holidays

for i in HOLIDAYS:
    create_holidays(YEAR,MONTH, i)

# print(holidays)

[datetime.date(2020, 12, 10), datetime.date(2020, 12, 11), datetime.date(2020, 12, 31)]


In [6]:
# remove the holidays and unwanted weekdays

In [7]:
def remove_dates(all_dates, unwanted_weekdays, holidays): #unwanted = list-of-list of unwanted weekdays
#     OFM = all_dates- sundays - saturdays - tuesdays 
    OFM = all_dates
    for weekday in unwanted_weekdays:
        OFM = [date for date in OFM if date not in weekday_date[weekday]]
    OFM = [date for date in OFM if date not in holidays]
    return OFM

unwanted_weekdays = ['sunday','saturday','tuesday']
OFM_dates = remove_dates(all_dates, unwanted_weekdays, holidays)

print('Total number of days needed to fill =', len(OFM_dates), '\n')
print('The OFM dates are:')
OFM_dates

Total number of days needed to fill = 15 

The OFM dates are:


[datetime.date(2020, 12, 2),
 datetime.date(2020, 12, 3),
 datetime.date(2020, 12, 4),
 datetime.date(2020, 12, 7),
 datetime.date(2020, 12, 9),
 datetime.date(2020, 12, 14),
 datetime.date(2020, 12, 16),
 datetime.date(2020, 12, 17),
 datetime.date(2020, 12, 18),
 datetime.date(2020, 12, 21),
 datetime.date(2020, 12, 23),
 datetime.date(2020, 12, 24),
 datetime.date(2020, 12, 25),
 datetime.date(2020, 12, 28),
 datetime.date(2020, 12, 30)]

### Get the list of intern doctors' names
The names are stored in a csv file(exported from Excel)


In [21]:
opened_file = open('Book1.csv')
read_file = pd.read_csv("Book1.csv", encoding = 'utf-8' )
data = read_file.iloc[:,0:2]

# data

The names looked like this : 
>Int3 ภาณุวัฒน์, Med

So we need to separate the year indicator (Int3 indicates intern year 3) from the name. 

In [22]:
year_list = []
name_list = []
index_list = list(range(len(data)))
# splitting int from name
for i in read_file.iloc[:,0]:
    year,name = i.split()
    name = name.replace(' ','')
    year_list.append(year)
    name_list.append(name)
    
    
# create a pd with intern year as column and insert it back as first column in the main file (df)
df = {'Index': index_list,
      'Year': year_list,
      'Name':name_list} 
df = pd.DataFrame(df) 
df.insert(2, 'Fix', data['Fix'], True) 
L_df = df.copy(deep=True) # แก้ Bug คนที่ลาแล้วไม่มี year นำหน้า

# df

In [23]:
# Create dictionary for later use
index_to_name = {i:n for i,n in enumerate(df['Name'])}

# index_to_name

### Remove names

* Intern-1s aren't included in OFM shifts, so we need to remove them
* Remove resigned
* Remove those who are not present at the hospital for this month

In [11]:
# remove intern 1
df.drop(df[df.loc[:,'Year'] == 'Int1'].index, inplace = True)
# remove NOT_PRESENT + RESIGNED + OTHER_OPD
for name in (NOT_PRESENT + RESIGNED + OTHER_OPD):
    df.drop(df[df.loc[:,'Name'] == name].index, inplace = True)

print("Resigned :{}".format(RESIGNED), "Not_preset :{}".format(NOT_PRESENT), "Other OPD: {}".format(OTHER_OPD))
print("There are {0} interns present, มีอินเทิร์นเหลือทั้งหมด {0} คน".format(len(df)))
df

Resigned :['ศุภาพิชญ์', 'พุฒิพงศ์'] Not_preset :['ภาณุวัฒน์', 'ปรียาภรณ์'] Other OPD: []
There are 17 interns present, มีอินเทิร์นเหลือทั้งหมด 17 คน


,Index,Year,Fix,Name
10,10,Int2,Sx,ณัฐชัย
15,15,Int3,Ortho,ภาสกร
16,16,Int2,Ortho,ปารินทร์
19,19,Int3,Ped,เอนกโชค
20,20,Int3,Ped,ณัชชา
21,21,Int2,Ped,ชญานิศ
22,22,Int2,Ped,นภสร
23,23,Int2,Ped,ณิชารีย์
24,24,Int2,Ped,ปภาวดี
31,31,Int3,Eye,นัทธมน


In [12]:
import pickle
def try_open_saves(month):
    month_file = str(month).lower()+'_count'
    print('searching for',month_file,'..retrieving data..','\n')
        
    try:
        with open(month_file, 'rb') as f:
            old_dict = pickle.load(f)   
            for key in old_dict:
                print('The number of shifts so far...{}..x..{}..times'.format(key, old_dict[key]))
    except:
        old_dict = '{}'
        print('No roster was made before, nothing to display - ยังไม่เคยมีการจัดเวรมาก่อน')
    return old_dict

print('loading saves..')
old_dict = try_open_saves(PREVIOUS_SAVE)

loading saves..
searching for november_count ..retrieving data.. 

The number of shifts so far...นภสร..x..6..times
The number of shifts so far...พิชชาภา..x..6..times
The number of shifts so far...ชญานิศ..x..6..times
The number of shifts so far...สรศักย์..x..6..times
The number of shifts so far...อภิชญา..x..6..times
The number of shifts so far...ณัฐชัย..x..6..times
The number of shifts so far...ณิชารีย์..x..6..times
The number of shifts so far...ภาสกร..x..6..times
The number of shifts so far...บุญฤทธิ์..x..4..times
The number of shifts so far...ภาณุวัฒน์..x..5..times
The number of shifts so far...ณัชชา..x..6..times
The number of shifts so far...นัทธมน..x..5..times
The number of shifts so far...ณัฐภัสสร..x..6..times
The number of shifts so far...เอนกโชค..x..6..times
The number of shifts so far...ปภาวดี..x..6..times
The number of shifts so far...ปารินทร์..x..5..times
The number of shifts so far...พุฒิพงศ์..x..6..times
The number of shifts so far...ปรียาภรณ์..x..1..times
The number of shif

In [13]:
import time
import sys
import random

class unavailable:
    def __init__(self, UA_input, weekday, df=df, weekday_date=weekday_date):
        # weekday could be str (eg.'monday') or be integer (eg. 28)   
        if type(weekday) == str: 
            self.dates = [i.day for i in weekday_date[weekday]]
        elif (type(weekday) == list)& (type(weekday[0])==str):
            self.dates = []
            for i in weekday:
                a = [c.day for c in weekday_date[i]]
                self.dates += a
        else:
            self.dates = weekday
            
        # UA_input could be  (subspecialty ('Fix')) or list of names
        if UA_input in df.loc[:,'Fix'].unique():
            self.names = [i for i in df[df.loc[:,'Fix'] == UA_input].Name]
        else: 
            self.names = UA_input
            
        self.df = df
        self.weekday_date = weekday_date
        print ('The unavailable dates for {} are day {}'.format(self.names, self.dates))
                
    def UA_bool(self, random_date, random_winner):
        # Will return False if the person is unavailable at a given date  
        if (random_winner in self.names) and (random_date in self.dates):
            return False
        else:
            return True
        
print('checking unavailable Interns...') 
print("Not_preset :{}".format(NOT_PRESENT), "Other OPD: {}".format(OTHER_OPD))
print("There are {0} interns present,".format(len(df)),'Number of days to be filled = {}'.format(len(OFM_dates)))
print('checking unavailable dates...')  

var_list = []
for n,(who,when) in enumerate(UNAVAILABLE):
    var_name = "who_" + str(n)
    vars()[var_name] = unavailable(who,when) #     declare var name with string 
    var_list.append(eval(var_name)) #     evaluate string as if it's a var
    
def evaluate_conditions(random_date,random_winner):
    conditions = []
    for i in var_list:
        conditions.append(i.UA_bool(random_date,random_winner))
    yield conditions                                                                     

####### start randomization #######

print('\nInitiating randomization..')
"""change every month"""
print('\nเวร OFM ' + Thai_dict[SAVE.lower()])

count_dict= {}
day_name_dict = {}
start_time = time.time()
seconds = 0
repeat = None
while len(day_name_dict) < len(OFM_dates):
    for date in OFM_dates:
        if (date.day not in day_name_dict.keys()):
            random_winner = random.choice(df.loc[:,'Name'].values)
            current_time = time.time()
            elapsed_time = current_time - start_time
            if (random_winner not in day_name_dict.values())and all(*evaluate_conditions(date.day,random_winner)):
                day_name_dict[date.day] = random_winner
                count_dict[random_winner] = 1
        
            elif elapsed_time > 0.5 and all(*evaluate_conditions(date.day,random_winner)):
                """FIX คนที่อยู่ 2 วัน, just comment it out, when not in use"""
                # random_winner = random.choice( ["นภสร","ณัชชา"])
                """"""""""""""""""
                day_name_dict[date.day] = random_winner
                count_dict[random_winner] += 1
                name_day = {v: [k for k in day_name_dict if day_name_dict[k] == v] for v in set(day_name_dict.values())}
                repeat = [i for i in name_day if len(name_day[i]) >= 2]
                
    """restart if these names are not in the dictionary"""           
    # if not(all(x in day_name_dict.values() for x in ["นภดล",'ศิรวิชญ์','ปรียาภรณ์'])):
    # day_name_dict = {}

if repeat != None:
    print("\n",'คนที่โดนอยู่มากกว่า 1 วันได้แก่', "\n")
    for i in repeat:
        print("{}..อยู่วันที่..{}".format(i,sorted(name_day[i])))
print("\n")

import collections
day_name_dict = collections.OrderedDict(sorted(day_name_dict.items(), key=lambda t: t[0])) # sort dictionary using ordered dict, and sort by key (t[0])

for i in day_name_dict:
    year = df[df.loc[:,'Name'] == day_name_dict[i]].Year.values
    name = day_name_dict[i]
    print('Day..{}..:.{} {}'.format(i,year, name))

checking unavailable Interns...
Not_preset :['ภาณุวัฒน์', 'ปรียาภรณ์'] Other OPD: []
There are 17 interns present, Number of days to be filled = 15
checking unavailable dates...
The unavailable dates for ['สรศักย์'] are day [3, 10, 17, 24, 31]
The unavailable dates for ['ภาณุวัฒน์'] are day [2, 9, 16, 23, 30]
The unavailable dates for ['ศิรวิชญ์'] are day [7, 14, 21, 28, 3, 10, 17, 24, 31]
The unavailable dates for ['ปรียาภรณ์'] are day [2, 9, 16, 23, 30, 4, 11, 18, 25]
The unavailable dates for ['นภดล'] are day [1, 8, 15, 22, 29, 3, 10, 17, 24, 31]

Initiating randomization..

เวร OFM ธันวาคม


Day..2..:.['Int2'] นภดล
Day..3..:.['Int3'] บุญฤทธิ์
Day..4..:.['Int2'] ณัฐชัย
Day..7..:.['Int3'] นัทธมน
Day..9..:.['Int2'] อภิชญา
Day..14..:.['Int2'] ชญานิศ
Day..16..:.['Int2'] ณิชารีย์
Day..17..:.['Int3'] เอนกโชค
Day..18..:.['Int3'] ณัชชา
Day..21..:.['Int3'] ภาสกร
Day..23..:.['Int2'] ปภาวดี
Day..24..:.['Int2'] ปารินทร์
Day..25..:.['Int3'] ศิรวิชญ์
Day..28..:.['Int3'] ณัฐภัสสร
Day..30..:.['Int2

In [14]:
print('\n updating updated_dict \n')
import copy
updated_dict = copy.deepcopy(old_dict)
if old_dict != {}:
    for i in count_dict:
        if i in updated_dict:
            updated_dict[i]+= count_dict[i]
        else:
            updated_dict[i] = 0 
            updated_dict[i]+= count_dict[i]
elif old_dict == {}:
    updated_dict = count_dict

# print the counting in updated dict
for key in updated_dict:
    year = L_df[L_df.loc[:,'Name'] == key].Year.values
    print('The accumulated number of shifts so far..{} {}..x..{}..times'.format(year,key, updated_dict[key]))


 updating updated_dict 

The accumulated number of shifts so far..['Int2'] นภสร..x..6..times
The accumulated number of shifts so far..['Int2'] พิชชาภา..x..7..times
The accumulated number of shifts so far..['Int2'] ชญานิศ..x..7..times
The accumulated number of shifts so far..['Int2'] สรศักย์..x..6..times
The accumulated number of shifts so far..['Int2'] อภิชญา..x..7..times
The accumulated number of shifts so far..['Int2'] ณัฐชัย..x..7..times
The accumulated number of shifts so far..['Int2'] ณิชารีย์..x..7..times
The accumulated number of shifts so far..['Int3'] ภาสกร..x..7..times
The accumulated number of shifts so far..['Int3'] บุญฤทธิ์..x..5..times
The accumulated number of shifts so far..['Int3'] ภาณุวัฒน์..x..5..times
The accumulated number of shifts so far..['Int3'] ณัชชา..x..7..times
The accumulated number of shifts so far..['Int3'] นัทธมน..x..6..times
The accumulated number of shifts so far..['Int3'] ณัฐภัสสร..x..7..times
The accumulated number of shifts so far..['Int3'] เอนกโชค

### Define functions for saving, backup and reading the files

In [32]:
from os import path
import shutil

def save_file(file_name, dict):
    with open(file_name, 'wb') as f:
        pickle.dump(dict, f)
        return 'succesfully rewrite file {}'.format(file_name)

def backup(file_name):
    if path.exists(file_name):
        if path.exists(file_name+'_backup'):
            print('backup already existed, will not overwrite file')
        else:
            print('file exists, creating backup:', file_name+'_backup')
            shutil.copy(file_name, file_name+'_backup')
    else:
        print('file did not exist, will not create backup')
    
def read_count(file_name): 
    with open(file_name, 'rb') as f:
        file = pickle.load(f)    
        f.close()
        for key in file:
            print('The number of shifts so far...{} {}..x..{}..times'.format(L_df[L_df.loc[:,'Name'] == key].Year.values,key, file[key]))
        
def read_date(file_name):
    with open(file_name, 'rb') as f:
        file = pickle.load(f)    
        f.close()
        for key in file:
            year = L_df[L_df.loc[:,'Name'] == file[key]].Year.values
            print('Day..{}..:..{} {}'.format( key,year,file[key]))

In [33]:
# save to 2 files(_date and _count) and create backup if intending to overwrite file
# day_name_dict is currently orderred dict -> Change back to normal dict- else not readable
def save_all(month):
    date_file = str(month).lower() + '_date'
    count_file = str(month).lower() + '_count'
    try: 
        print('try saving _date file: {}'.format(date_file))
        backup(date_file)
        save_file(date_file, day_name_dict)
        print('success')
    except: 
        print('failed')
        
    try: 
        print('try saving _count file {}'.format(count_file))
        backup(count_file)
        save_file(count_file, updated_dict)
        print('success')
    except: 
        print('failed')

#Read both the count and date files -- just enter the month
def read_all(month):
    date_file = str(month).lower() + '_date'
    count_file = str(month).lower() + '_count'
    print('\n reading {} \n'.format(date_file))
    read_date(date_file)
    print('\n reading {} \n'.format(count_file))
    read_count(count_file)

save_all(SAVE)
read_all(SAVE)

try saving _date file: december_date
file did not exist, will not create backup
success
try saving _count file december_count
file did not exist, will not create backup
success

 reading december_date 

Day..2..:..['Int2'] นภดล
Day..3..:..['Int3'] บุญฤทธิ์
Day..4..:..['Int2'] ณัฐชัย
Day..7..:..['Int3'] นัทธมน
Day..9..:..['Int2'] อภิชญา
Day..14..:..['Int2'] ชญานิศ
Day..16..:..['Int2'] ณิชารีย์
Day..17..:..['Int3'] เอนกโชค
Day..18..:..['Int3'] ณัชชา
Day..21..:..['Int3'] ภาสกร
Day..23..:..['Int2'] ปภาวดี
Day..24..:..['Int2'] ปารินทร์
Day..25..:..['Int3'] ศิรวิชญ์
Day..28..:..['Int3'] ณัฐภัสสร
Day..30..:..['Int2'] พิชชาภา

 reading december_count 

The number of shifts so far...['Int2'] นภสร..x..6..times
The number of shifts so far...['Int2'] พิชชาภา..x..7..times
The number of shifts so far...['Int2'] ชญานิศ..x..7..times
The number of shifts so far...['Int2'] สรศักย์..x..6..times
The number of shifts so far...['Int2'] อภิชญา..x..7..times
The number of shifts so far...['Int2'] ณัฐชัย..x..7.

แก้ตารางด้วยตัวเองทีหลัง

In [29]:
##### edit roster manually #####
# with open(str("November").lower() + '_date', 'rb') as f:
#     file = pickle.load(f)    
#     f.close()
# file

In [17]:
# day_name_dict = dict([(2, 'พุฒิพงศ์'),
#              (4, 'ณัชชา'),
#              (5, 'นภสร'),
#              (6, 'ชญานิศ'),
#              (9, 'ณิชารีย์'),
#              (11, 'อภิชญา'),
#              (12, 'ณัฐชัย'),
#              (13, 'ปภาวดี'),
#              (16, 'นัทธมน'),
#              (18, 'เอนกโชค'),
#              (23, 'นภดล'),
#              (25, 'ศิรวิชญ์'),
#              (26, 'พิชชาภา'),
#              (27, 'ปรียาภรณ์'),
#              (30, 'ณัฐภัสสร')])
# 
# try: 
#     save_file(str("November").lower() + '_date', day_name_dict)
#     print('success')
#     read_date(str("November").lower() + '_date')
# except: 
#     print('failed')

In [30]:
##### edit count file manually #####
# with open(str("November").lower() + '_count', 'rb') as f:
#     file = pickle.load(f)    
#     f.close()
# file

In [31]:
### เดือนที่แล้วพุฒิพงศ์อยู่แทนเอนกโชค ###
# updated_dict["พุฒิพงศ์"] += 1
# updated_dict["เอนกโชค"] -= 1
# updated_dict
# 
# try: 
#     save_file(str("November").lower() + 'count', updated_dict)
#     print('success')
#     read_count(str("November").lower() + '_count')
# except: 
#     print('failed')